In [94]:
import requests
import uuid

# ПОЛУЧАЕМ ДАННЫЕ О СПИСКЕ КОЛЛЕКЦИЙ

In [99]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "courses", "data_key": "","action": "get_data" }
resp = requests.post("http://localhost:8000/api/add_message", json=message)
resp

<Response [200]>

In [101]:
message = { "id": id }
resp = requests.post("http://localhost:8000/api/get_message_result", json=message)
resp.json()

{'message': {'id': '5eb7c9e3-6fd8-4fb5-886a-a7956511edaa',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'courses',
  'data_key': '',
  'action': 'get_data',
  'timestamp': 1669464208},
 'result': {'courses': ['programming', 'test']}}

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ

In [118]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "programming", "action": "get_data" }
resp = requests.post("http://localhost:8000/api/add_message", json=message)
resp

<Response [200]>

In [121]:
message = { "id": id }
resp = requests.post("http://localhost:8000/api/get_message_result", json=message)
resp

<Response [200]>

In [122]:
resp.json()

{'message': {'id': '5c27a3dd-a932-4912-8409-0534558795d0',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'problems',
  'data_key': 'programming',
  'action': 'get_data',
  'timestamp': 1669465071},
 'result': {'problems': [{'1': ['1', '2'], 'task': ''},
   {'2': ['1', '2'], 'task': ''},
   {'3': ['1'],
    'task': 'Задача - написать алгоритм, который получает на вход целое число и возвращает число на 1 большее.'}]}}